# Basic Text Regressor using Auto-Keras
test for proposed scores: the best for the validation data is :

df['score2']=0.16*df['obscene']+0.32*df['toxic']+1.5*df['threat']+0.64*df['insult']+1.5*df['severe_toxic']+1.5*df['identity_hate']

notes:
since bert fast text are not working for autokeras in this environment: this is not a competitive kernel :)


In [ ]:
import sys
package_dir = '../input/autokeras-april-2021'
sys.path.insert(0, package_dir)

In [ ]:
from scipy.stats import rankdata

In [ ]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

import autokeras as ak

In [ ]:
df = pd.read_csv("../input/jigsaw-score-augumentation/jigsaw_train_hate_annotationprob.csv",index_col=0)
#df=df.sample(1000)

In [ ]:
df.head()

In [ ]:
class_1,class_2 = df.toxic.value_counts()
print(class_1,class_2)
#class_4,class_3,class_2,class_1 = df.label.value_counts()
#c2 = df[(df.toxic ==1 ) | (df.severe_toxic== 1) | (df.obscene ==1) | (df.insult == 1) | (df.identity_hate == 1)]
#c2 = df[(df.toxic > 0 )]
c2 = df[df["toxic"] >0]
c1 = df[df.toxic == 0]
print(len(c1),len(c2))
df_2 = c1.sample(class_2)
print(len(df_2))
undersampled_df = pd.concat([df_2,c2],axis=0)
undersampled_df.drop_duplicates(subset ="text",keep = False, inplace = True)
df=undersampled_df
del undersampled_df
del c1
del c2

In [ ]:
#df.sample(frac=0.5)

In [ ]:
#df.head()

In [ ]:
#scores_df=pd.read_csv("../input/jigsaw-score-augumentation/jigsaw_train_hate_annotationprob.csv",index_col=0)
#scores_df=scores_df.sample(1000)

In [ ]:
df['score1']=10*df['jigsaw_df_severity']*df['offensive_agreement_rating']
df['score2']=0.16*df['obscene']+0.32*df['toxic']+1.5*df['threat']+0.64*df['insult']+1.5*df['severe_toxic']+1.5*df['identity_hate']
df['score3']=1*df['obscene']+1*df['toxic']+1*df['threat']+1*df['insult']+2*df['severe_toxic']+1.5*df['identity_hate']
df['score4']=1*df['obscene']+1*df['toxic']+1*df['threat']+1*df['insult']+2*df['severe_toxic']+1*df['identity_hate']
df['score5']=10*df['jigsaw_df_severity']*df['offensive_agreement_rating']+10*df['jigsaw_df_severity']*df['hate_speech_agreement']
df['score6']=1*df['obscene']+1*df['toxic']+1*df['insult']+5*df['jigsaw_df_severity']*df['offensive_agreement_rating']+5*df['jigsaw_df_severity']*df['hate_speech_agreement']
df['score7']=1*df['obscene']+1*df['toxic']+1*df['threat']+1*df['insult']+1.5*df['severe_toxic']+1*df['identity_hate']
df['score8']=10*df['offensive_agreement_rating']+10*df['hate_speech_agreement']
df['score9']=0.2*df['obscene']+0.4*df['toxic']+0.6*df['threat']+0.2*df['insult']+0.9*df['severe_toxic']+0.9*df['identity_hate']

In [ ]:
#df.columns

In [ ]:
scores_df= df[['score1','score2', 'score3', 'score4', 'score5', 'score6', 'score7', 'score8','score9']]

In [ ]:
column_names=['score1','score2', 'score3', 'score4', 'score5', 'score6', 'score7', 'score8','score9']

In [ ]:
#scores_df.describe()

In [ ]:
#df.tail(25)

In [ ]:
#df.describe()

In [ ]:
#df['target']=df['proposed_score3']+0.1*df['offensive_agreement_rating']

In [ ]:
#df= df[['text', 'target']]

In [ ]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.10)
#valid, test = train_test_split(valid, test_size=0.3)

In [ ]:

#valid.loc[:,['score1','score2', 'score3', 'score4', 'score5', 'score6', 'score7']]

In [ ]:
df_test=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv",index_col=0)

In [ ]:
comments_to_score=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv",index_col=0)

In [ ]:
X_less_toxic =df_test.less_toxic.values
X_more_toxic =df_test.more_toxic.values
X_comments_to_score =comments_to_score.text.values

In [ ]:
train_data = train['text'].values
train_target = train.loc[:,column_names].values

valid_data = valid['text'].values
valid_target = valid.loc[:,column_names].values

print(train_data.shape, train_target.shape)
print(valid_data.shape, valid_target.shape)

Simple model 0.6745383286834064 

* validation score with second conv block and 3 epoch:
0.6651720472963997


In [ ]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
# Use separable Conv layers in Keras.
output_node = ak.ConvBlock(separable=True)(output_node)
#output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=5
)
reg.fit(train_data, train_target, epochs=5,validation_split=0.15)

In [ ]:
p1 = reg.predict(X_less_toxic).squeeze().tolist()

In [ ]:
p2 = reg.predict(X_more_toxic).squeeze().tolist()

In [ ]:
p1[100][8]

In [ ]:
# #preds = reg.predict(X_less_toxic).squeeze().tolist()
# #p1 = predictor.predict(X_less_toxic)
# p1 = reg.predict(X_less_toxic).squeeze().tolist()
# #p2 = predictor.predict(X_more_toxic)
# p2 = reg.predict(X_more_toxic).squeeze().tolist()
# #\

In [ ]:
def compare(p1,p2):
    z=0
    k=0
    n=len(p1)
    for i in range(len(p1)):
        if p1[i]< p2[i]:
            z=z+1
        else:
            k=k+1
            #print('nu')
    print(z/n)

    return z/n

In [ ]:
def compare_list(p1,p2):

    for j in range(9):
            z=0
            k=0
            n=len(p1)
            print("score",j+1,"score",j+1,)
            for i in range(len(p1)):
                if p1[i][j]< p2[i][j]:
                    z=z+1
                else:
                    k=k+1
                #print('nu')
            print(z/n)
            #print("p1 score",j,"p2 score",j,z/n)
    return z/n

In [ ]:
compare_list(p1,p2)

In [ ]:
# # Validation Accuracy
# compare(p1,p2)

In [ ]:
# p1 = predictor.predict(X_less_toxic)
# p2 = predictor.predict(X_more_toxic)
# # Validation Accuracy
# (p1 < p2).mean()

In [ ]:
# Get the predictions 
scored_sub=reg.predict(X_comments_to_score).squeeze().tolist()

In [ ]:
len(scored_sub)

In [ ]:
len(comments_to_score)

In [ ]:
comments_to_score['scores']=scored_sub

In [ ]:
scores=pd.DataFrame(comments_to_score['scores'].to_list(), columns=['score1','score2', 'score3', 'score4', 'score5', 'score6', 'score7', 'score8', 'score9'])

In [ ]:
scores.head()

In [ ]:
colnames=scores.columns

In [ ]:
for col in colnames:
    rank="rank"+col
    comments_to_score[rank]=rankdata(scores[col], method='ordinal')
    #print(rank)

In [ ]:
comments_to_score.head()

In [ ]:
# #comments_to_score['score']=scored_sub
# comments_to_score['rank']=rankdata(scored_sub, method='ordinal')

In [ ]:
comments_to_score.columns

In [ ]:
#comments_to_score.columns
cols=['rankscore1', 'rankscore2', 'rankscore3',
       'rankscore4', 'rankscore5', 'rankscore6', 'rankscore7', 'rankscore8',
       'rankscore9']

In [ ]:
comments_to_score['score'] = comments_to_score[cols].sum(axis=1)

In [ ]:
comments_to_score['grand_slam_ranks']=rankdata( comments_to_score['score'], method='ordinal')

In [ ]:
submission=comments_to_score.loc[:,['grand_slam_ranks']]

In [ ]:
submission.rename(columns={"grand_slam_ranks": 'score'}, inplace = True)

In [ ]:
submission.to_csv('submission.csv')

In [ ]:
submission.head()

In [ ]:
 #scores.corr()